In [ ]:
from IPython.display import HTML
HTML(open('../style.css').read())

# The [Zebra Puzzle](https://en.wikipedia.org/wiki/Zebra_Puzzle)

The following puzzle appeared in the magazine *Life International* on the 17th of December in the year 1962:
* There are five houses.
* The Englishman lives in the red house.
* The Spaniard owns the dog.
* Coffee is drunk in the green house.
* The Ukrainian drinks tea.
* The green house is immediately to the right of the ivory house.
* The Old Gold smoker owns snails.
* Kools are smoked in the yellow house.
* Milk is drunk in the middle house.
* The Norwegian lives in the first house.
* The man who smokes Chesterfields lives in the house next to the man with the fox.
* Kools are smoked in the house next to the house where the horse is kept.
* The Lucky Strike smoker drinks orange juice.
* The Japanese smokes Parliaments.
* The Norwegian lives next to the blue house.

Furthermore, each of the five houses is painted in a different colour, their inhabitants are of different nationalities, own different pets, drink different beverages, and smoke different brands of cigarettes.

Our task is to answers the following questions: 
1. *Who drinks water?*
2. *Who owns the zebra?*

## Choosing the Appropriate Variables

In order to solve this problem we use the following propositional variables, where the counter $i$ ranges from $1$ to $5$.  

* $\texttt{English}\langle i \rangle$ expresses the fact
  that the Englishman lives in house number $i$.
  
  The remaining nationalities are coded using the variables
  $\texttt{Spanish}\langle i \rangle$, $\texttt{Ukrainian}\langle i \rangle$,
  $\texttt{Norwegian}\langle i \rangle$, and $\texttt{Japanese}\langle i \rangle$.
* $\mathtt{Red}\langle i \rangle$ expresses that house number $i$ is red.
  
  We use the variables $\texttt{Green}\langle i \rangle$, $\texttt{Ivory}\langle i \rangle$,
  $\texttt{Yellow}\langle i \rangle$, and $\texttt{Blue}\langle i \rangle$ to encode the remaining
  colors.
* $\texttt{OldGold}\langle i \rangle$ expresses that the inhabitant of house number $i$ smokes cigarettes of the brand "Old Gold".
  
  We use the variables $\texttt{Kools}\langle i \rangle$,
  $\texttt{Chesterfields}\langle i \rangle$, $\texttt{LuckyStrike}\langle i \rangle$, and
  $\texttt{Parliaments}\langle i \rangle$ to encode the cigarette brands.
* $\texttt{Dog}\langle i \rangle$ expresses the fact that the inhabitant of house number
  $i$ keeps a dog as his pet.

  We use the variables $\texttt{Snails}\langle i \rangle$, $\texttt{Fox}\langle i \rangle$,
  $\texttt{Horse}\langle i \rangle$, and $\texttt{Zebra}\langle i \rangle$ to encode the
  remaining pets.
* $\texttt{Coffee}\langle i \rangle$ expresses the fact the the inhabitant of house number $i$
  drinks coffee.

  We use the variables $\texttt{Milk}\langle i \rangle$,
  $\texttt{OrangeJuice}\langle i \rangle$, $\texttt{Tea}\langle i \rangle$, 
  and $\texttt{Water}\langle i \rangle$ to encode the remaining drinks.

We are using the angular brackets "$\langle$" and "$\rangle$" as part of the variable names 
because our parser for propositional logic accepts these symbols as part of variable names.  The parser would be confused if we would use parentheses.

## Importing the Necessary Modules

Our goal is to solve this puzzle by first coding it as a solvability problem of propositional logic and then to solve the resulting set of clauses using the algorithm of Davis and Putnam.

In [ ]:
%%capture 
%run 07-Davis-Putnam-JW.ipynb

In order to be able to transform formulas from propositional logic into sets of clauses we import the module <tt>cnf</tt> which implements the function <tt>normalize</tt> that takes a formula and transforms it into a set of clauses.

In [ ]:
%%capture
%run 04-CNF.ipynb

In order to write formulas conveniently, we use the parser for propositional logic.

In [ ]:
%%capture
%run Propositional-Logic-Parser.ipynb

Using the parser and the module <tt>cnf</tt> we can impement a function $\texttt{parseKNF}(s)$ that takes a string $s$ representing a formula and transforms $s$ into an equivalent set of clauses.

In [ ]:
def parseKNF(s):
    nestedTuple = parse(s)
    Clauses     = normalize(nestedTuple)
    return Clauses

## Auxiliary Functions

In order to succinctly express the constraints that all houses have different colours, the inhabitants have different nationalities etc., it is convenient to implement a function $\texttt{atMostOne}(V)$ that takes a set of variables $V$ and returns a set of formulas that is true if and only if all the variables from $V$ have different values.

In [ ]:
def atMostOne(V):
    return { frozenset({('¬',p), ('¬', q)}) for p in V
                                            for q in V 
                                            if  p != q 
           }

Given a name $f$ and an index $i \in\{1,2,3,4,5\}$, the function $\texttt{var}(i)$ creates the string 
$f\langle i \rangle$, e.g. the call `var("Japanese", 2)` the following string:
```
Japanese<2>
```

In [ ]:
def var(f, i):
    return f"{f}<{str(i)}>" 

In [ ]:
var("Japanese", 2)

A call of the form $\texttt{somewhere}(x)$ will return a clause that specifies that the person with property $x$ has to live in one of the houses from $1$ to $5$.  In order to be able to insert this clause into a set, we have to make sure that we return a `frozenset`.

In [ ]:
def somewhere(x):
    return frozenset({ var(x, i) for i in range(1, 5+1) })

In [ ]:
somewhere('Japanese')

Given an exclusive set of properties $S$ and a house number $i$, the function $\texttt{atMostOne}(S, i)$ returns a set of clauses that specifies that the person living in house number $i$ has at most one of the properties from the set $S$.  For example, if 
$S = \{\texttt{"Japanese"}, \texttt{"Englishman"}, \texttt{"Spaniard"}, \texttt{"Norwegian"}, \texttt{"Ukranian"}\}$, 
then $\texttt{atMostOne}(S, 3)$ specifies that the inhabitant of house number 3 has at most one of the nationalities from the set $S$.

In [ ]:
def atMostOneAt(S, i):
    return atMostOne({var(x, i) for x in S})

In [ ]:
atMostOneAt({"A", "B", "C"}, 1)

The function $\texttt{onePerHouse}(S)$ is called as follows:
$$\texttt{onePerHouse}(\{\texttt{"Japanese"},
       \texttt{"English"}, 
       \texttt{"Spanish"}, \texttt{"Norwegian"}, 
       \texttt{"Ukrainian"}\})
$$
This function creates a set of clauses that expresses that there has to be a house where the Japanese lives, a house where the Englishman lives, a house where the Spaniard lives, a house where the Norwegian lives, and a house
where the Ukranian lives.  Furthermore, the set of clauses would contain clauses that express that these five persons live in **different** houses.

In [ ]:
def onePerHouse(S):
    Clauses  = { somewhere(x) for x in S } 
    Clauses |= { C for i in range(1, 5+1) for C in atMostOneAt(S, i) }
    return Clauses

In [ ]:
onePerHouse({"A", "B"})

Given two properties $a$ and $b$ the function $\texttt{sameHouse}(a, b)$ computes a set of clauses that specifies that if the inhabitant of house number $i$ has the property $a$, then he also has the the property $b$ and vice versa.  For example, $\texttt{sameHouse}(\texttt{"Japanese"}, \texttt{"Dog"})$ specifies that the Japanese guy keeps a dog.

In [ ]:
def sameHouse(a, b):
    return { C for i in range(1,5+1)
               for C in parseKNF(f"{var(a, i)} ↔ {var(b, i)}") 
           }

In [ ]:
sameHouse("Red", "Tea")

Given to properties $a$ and $b$ the function $\texttt{nextTo}(a, b)$ computes a set of clauses that specifies that the inhabitants with properties $a$ and $b$ are direct neighbours.  For example, $\texttt{nextTo}(\texttt{'Japanese'}, \texttt{'Dog'})$ specifies that the Japanese guy lives next to the guy who keeps a dog.

In [ ]:
def nextTo(a, b):
    Result  = parseKNF(f"{var(a,1)} → {var(b,2)}")
    Result |= { C for i in [2,3,4] for C in parseKNF(f"{var(a,i)} → {var(b,i-1)} ∨ {var(b,i+1)}") }
    Result |= parseKNF(f"{var(a,5)} → {var(b,4)}")
    return Result

In [ ]:
nextTo('A', 'B')

Given to properties $a$ and $b$ the function $\texttt{leftTo}(a, b)$ computes a list of clauses that specifies that the inhabitants with properties $a$ lives in the house to the left of the inhabitant who has property $b$.  For example, $\texttt{livesTo}(\texttt{'Japanese'}, \texttt{'Dog'})$ specifies that the Japanese guy lives in the house to the left of the house where there is a dog.

In [ ]:
def leftTo(a, b):
    Result  = { C for i in range(1, 5) for C in parseKNF(f"{var(a,i)} ↔ {var(b,i+1)}") }
    Result |= parseKNF(f"¬{var(a,5)}")
    return Result

In [ ]:
leftTo('A', 'B')

In [ ]:
Nations  = { "English", "Spanish", "Ukrainian", "Norwegian", "Japanese" }
Drinks   = { "Coffee", "Tea", "Milk", "OrangeJuice", "Water" }
Pets     = { "Dog", "Snails", "Horse", "Fox", "Zebra" }
Brands   = { "LuckyStrike", "Parliaments", "Kools", "Chesterfields", "OldGold" }
Colours  = { "Red", "Green", "Ivory", "Yellow", "Blue" }

The function `allClauses` returns a set of clauses describing the problem.

In [ ]:
def allClauses():
    # Every house has exactly one inhabitant.  This inhabitant has exactly one
    # nationality, one pet, smokes one brand of cigarettes, and has one type
    # of drink.  Furthermore, every house has exactly one color.
    Clauses  = onePerHouse(Nations)
    Clauses |= onePerHouse(Drinks)
    Clauses |= onePerHouse(Pets)
    Clauses |= onePerHouse(Brands)
    Clauses |= onePerHouse(Colours)
    # The Englishman lives in the red house.
    Clauses |= sameHouse("English", "Red")
    # The Spaniard owns the dog.
    Clauses |= sameHouse("Spanish", "Dog")
    # Coffee is drunk in the green house.
    Clauses |= sameHouse("Coffee", "Green")
    # The Ukrainian drinks tea.
    Clauses |= sameHouse("Ukrainian", "Tea")
    # The green house is immediately to the right of the ivory house.
    Clauses |= leftTo("Ivory", "Green")
    # The Old Gold smoker owns snails.
    Clauses |= sameHouse("OldGold", "Snails")
    # Kools are smoked in the yellow house.
    Clauses |= sameHouse("Kools", "Yellow")
    # Milk is drunk in the middle house.
    Clauses |= parseKNF("Milk<3>")
    # The Norwegian lives in the first house.
    Clauses |= parseKNF("Norwegian<1>")
    # The man who smokes Chesterfields lives in the house next 
    # to the man with the fox.
    Clauses |= nextTo("Chesterfields", "Fox")
    # Kools are smoked in the house next to the house where the horse is kept.
    Clauses |= nextTo("Kools", "Horse")
    # The Lucky Strike smoker drinks orange juice.
    Clauses |= sameHouse("LuckyStrike", "OrangeJuice")
    # The Japanese smokes Parliaments.
    Clauses |= sameHouse("Japanese", "Parliaments")
    # The Norwegian lives next to the blue house.
    Clauses |= nextTo("Norwegian", "Blue")
    return Clauses

In [ ]:
Clauses = allClauses()
Clauses

In [ ]:
len(Clauses)

In [ ]:
def main():
    Clauses = allClauses()
    return solve(Clauses)

Solving the problem takes about 0.1 seconds on my computer.

In [ ]:
%unload_ext nb_mypy

In [ ]:
%%time
Solution = main()

## Functions to PrettyPrint the Solution

In [ ]:
def arb(S):
    for x in S:
        return x

In [ ]:
def extractAssignment(Solution):
    Assignment = {}
    for Unit in Solution:
        Literal = arb(Unit)
        if isinstance(Literal, str):
            number = int(Literal[-2])
            name   = Literal[:-3]
            Assignment[name] = number
    return Assignment

In [ ]:
extractAssignment(Solution) # type: ignore

We need to import the functions `display` and `HTML` from `IPython.display` in order to be able to present the solution graphically. 

In [ ]:
from IPython.display import display, HTML

In [ ]:
def showHTML(Solution):
    Assignment = extractAssignment(Solution)
    result  = '<table style="border:2px solid blue">\n'
    result += '<tr>'
    for name in ['House', 'Nationality',  'Drink', 'Animal', 'Brand', 'Colour']:
        result += '<th style="color:gold; background-color:blue">' + name + '</th>'
    result += '</tr>\n'
    for chair in range(1, 5+1):
        result += '<tr><td style="border:1px solid green">' + str(chair) + '</td>'
        for Class in [Nations, Drinks, Pets, Brands, Colours]:
            for x in Class:
                if Assignment[x] == chair:
                    result += '<td  style="border:1px solid green">' + x + '</td>'
        result += '</tr>\n'
    result += '</table>'
    display(HTML(result))

In [ ]:
showHTML(Solution)

## Checking the Uniqueness of the Solution

Given a set of unit clauses $U$, the function $\texttt{negateSolution}(U)$ returns a clause that is the logical negation of $U$.

In [ ]:
Solution

In [ ]:
def negateSolution(UnitClauses):
    return { complement(arb(unit)) for unit in UnitClauses }

In [ ]:
negateSolution({frozenset({'a'}), frozenset({('¬', 'b')})})

In [ ]:
def checkUniqueness(Solution, Clauses):
    negation = negateSolution(Solution)
    Clauses.add(frozenset(negation))
    alternative = solve(Clauses)
    if alternative == { frozenset() }:
        print("The solution is unique!")
    else:
        print("ERROR: The solution is not unique!")

In [ ]:
%%time
checkUniqueness(Solution, Clauses)